# 대통령 연설문 네트워크 분석 코드

# 1. 필요한 라이브러리 다운로드 & 설치, 폰트 설정

### 필요한 라이브러리 설치

In [ ]:
# NLTK
#!pip install nltk
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

# 기타 라이브러리
import pandas as pd
import numpy as np
from IPython.display import display, HTML

from collections import defaultdict
from collections import Counter

# 한글형태소분석기: 키위 형태소분석기 설치
# 다양한 한글형태소분석기 비교: https://hipster4020.tistory.com/184
#!pip install --upgrade pip
#!pip install konlpy
#!pip install kiwipiepy  # 키위 형태소분석기 설치

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
stopwords = Stopwords()                # 불용어 처리시 사용

### 폰트

In [ ]:
# 시각화 라이브러리: seaborn, matplotlib
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 윈도우 시스템에 있는 Malgun Gothic 폰트를 사용
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 2. 빈도분석

In [ ]:
# ==================== 대통령 이름(for문으로 순회하여 코드 활용 가능)
name_file = ['YoonBoSun', 'ParkJeongHee', 'ChoiGyuHa', 'JeonDuHwuan',
             'RohTaeWoo', 'KimYoungSam', 'KimDaeJung', 'RohMooHyun', 'LeeMyungBak',
             'ParkGeunHye', 'MoonJaeIn', 'YoonSeokYeol']

name = "" # name_file에서 원하는 대통령 이름을 찾아 넣으십시오.

# ==================== 데이터 가져오기
file_path = f"./dataset/cleaned_data/cleaned_{name}.csv"
df_data = pd.read_csv(file_path)

# 선택하고자 하는 열 이름들을 리스트로 작성
selected_columns = ["title", "speech"]
df = df_data[selected_columns]

# ======================================== 전처리
# 전처리: 데이터 특수문자 제거
df['title_preprocessed'] = df['title'].str.replace('\n\n', ' ', regex=True).str.strip()
df['title_preprocessed'] = df['title_preprocessed'].str.replace('\n', ' ', regex=True).str.strip()
df['title_preprocessed'] = df['title_preprocessed'].str.replace('\\[.*?\\]', '', regex=True)
df['title_preprocessed'] = df['title_preprocessed'].str.replace('[^\\w\\s]', '', regex=True) # 특수 문자 삭제
df['title_preprocessed'] = df['title_preprocessed'].str.replace('625', '6.25', regex=True)

df['speech_preprocessed'] = df['speech'].str.replace('\n\n', ' ', regex=True).str.strip()
df['speech_preprocessed'] = df['speech_preprocessed'].str.replace('\n', ' ', regex=True).str.strip()
df['speech_preprocessed'] = df['speech_preprocessed'].str.replace('\\[.*?\\]', '', regex=True)
df['speech_preprocessed'] = df['speech_preprocessed'].str.replace('[^\\w\\s]', '', regex=True) # 특수 문자 삭제
df['speech_preprocessed'] = df['speech_preprocessed'].str.replace('625', '6.25', regex=True)


# ======================================== 형태소 분석 -> 제목&연설문 키워드
# 형태소 분석기 초기화
kiwi = Kiwi()

# 불용어 목록 정의
stopwords = {'와', '이', '에', '을', '고', '있', '습니다', '이'}

# 키워드 추출 함수 정의
def extract_keywords(text):
    analysis = kiwi.analyze(text)
    # 분석 결과가 여러 개일 수 있으므로 첫 번째 결과만 사용
    res = analysis[0][0]
    return [word + ('다' if tag in ['VV', 'VA'] else '')  # 동사와 형용사에 '다' 추가
            for word, tag, _, _ in res
            if tag in ['NNG', 'NNP', 'VV', 'VA', 'MAG'] and word not in stopwords and len(word) > 1]     # MAG 일반 부  사

# 제목/연설문 키워드 추출
df['title_keyword'] = df['title_preprocessed'].apply(extract_keywords) # 제목 키워드
df['speech_keyword'] = df['speech_preprocessed'].apply(extract_keywords) # 연설문 키워드
df['total_keyword'] = df['title_keyword'] + df['speech_keyword'] # 제목 + 연설문 키워드


# ======================================== 전체 빈도수 파일로 저장
# Counter()로 빈도수 세기
title_keywords = [keyword for sublist in df['title_keyword'] for keyword in sublist]
speech_keywords = [keyword for sublist in df['speech_keyword'] for keyword in sublist]
total_keywords = title_keywords + speech_keywords
total_counts = Counter(total_keywords)
print("전체 토큰 수 :", sum(total_counts.values())) # 전체 토큰 수

# 전체 키워드 파일 다운로드
df_total = pd.DataFrame(list(total_counts.items()), columns=['키워드', '빈도수'])
df_total = df_total.sort_values(by='빈도수', ascending=False)           # 빈도수 기준으로 정렬
df_total.to_csv(f'{name}_total_counts.csv', encoding='utf-8-sig', index=False)  # to_csv()로

# 데이터프레임을 입력 -> 최빈도 50 키워드 추출 -> 데이터프레임 반환
def df_to_50_keywords(df_data):
  """데이터프레임을 입력으로 하면 최빈도 50 키워드 추출하고 데이터프레임으로 반환하는 함수"""
  keywords = [keyword for sublist in df_data for keyword in sublist]         # keywords 전체 키워드 리스트
  counts = Counter(keywords)                                                 # counts 전체 키워드, 빈도수 튜플
  top_50_keywords = counts.most_common(50)                                   # 빈도수 상위 50 키워드

  # 데이터프레임으로 보기
  top_50_keywords = pd.DataFrame(top_50_keywords, columns=['키워드', '빈도수'])
  return top_50_keywords

# df_to_50_keywords()에 df['제목_키워드']를 입력해서 전체 키워드 빈도수 상위 50 추출
title_50_keywords = df_to_50_keywords(df['title_keyword'])
speech_50_keywords = df_to_50_keywords(df['speech_keyword'])
df_50_keywords = df_to_50_keywords(df['total_keyword']) # 상위 50개 빈도수 키워드/빈도수


# ======================================== 빈도수 상위 30개 그래프
# 빈도수 데이터 준비: '명사'를 index로, '빈도수'를 values로 하는 Series로 변환
series = pd.Series(data = df_50_keywords['빈도수'].values, index= df_50_keywords['키워드'])[:30]

# 시각화
plt.figure(figsize=(15, 6))
ax = sns.barplot(x=series.index, y=series.values, palette='viridis')
plt.xlabel('키워드', fontsize = 13)                       # 폰트사이즈 조절
plt.ylabel('빈도수', fontsize = 13)                       # 폰트사이즈 조절
plt.title('빈도수 상위 30위 키워드', fontsize = 17)       # 폰트사이즈, 그래프 타이틀 적절하게 수정!
plt.xticks(rotation=45, fontsize = 12)                    # 폰트사이즈 조절
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 각 막대 위에 기사 수 표시
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f'{int(height)}',
                xy=(p.get_x() + p.get_width() / 2, height),
                xytext=(0, 5),  # 5 points vertical offset
                textcoords='offset points',
                ha='center', va='bottom')

# 3. 연관어 분석

In [ ]:
import pandas as pd
import networkx as nx
from itertools import combinations
import re

#!pip install kss
import kss

In [6]:
# ======================================== [전처리 안된 문장, ..., 전처리 안된 문장]
sentences = [] # 문장 별 리스트

title = list(df['title'])
sentences.extend(title)
speeches = list(df['speech'])

# 연설문 문장을 리스트로 저장
"""
전처리 하기 전에 문장을 먼저 나눔.
sentence splitter가 더 잘 쪼개도록 만들기 위함임.
"""
for speech in speeches:
  sent = kss.split_sentences(speech)
  sentences.extend(sent)


# ======================================== [전처리된 문장, ..., 전처리된 문장]
cleaned_sentences = [] # 문장 속에서 전처리
for sentence in sentences:
    # 정규표현식 사용하여 처리
    cleaned = re.sub(r'\n\n', ' ', sentence).strip()
    cleaned = re.sub(r'\n', ' ', cleaned).strip()
    cleaned = re.sub(r'\\\[.*?\\\]', '', cleaned)  # 정규 표현식으로 대괄호 안의 내용을 제거
    cleaned = re.sub(r'[^\w\s]', '', cleaned)  # 특수 문자 제거
    cleaned = cleaned.replace('625', '6.25')  # 특정 문자열은 기본 replace 사용
    cleaned_sentences.append(cleaned)


# ======================================== [[키워드, ..., 키워드], ..., [키워드, ..., 키워드]]
sent_words_list = [] # 문장 별로 토크나이징

# 불용어 목록 정의
stopwords = {'와', '이', '에', '을', '고', '이'}

# 명사 추출 함수 정의
def extract_nouns(text):
    analysis = kiwi.analyze(text)
    # 분석 결과가 여러 개일 수 있으므로 첫 번째 결과만 사용
    res = analysis[0][0]
    return [word for word, tag, _, _ in res
            if tag in ['NNG', 'NNP'] and word not in stopwords and len(word) > 1]     # MAG 일반 부사

# 문장 단위로 토크나이징
for sentence in cleaned_sentences:
  sent_words_list.append(extract_nouns(sentence))

# 비어있는 리스트 제거 후 --> 최종 키워드 이중 리스트
final_sentences = [sentence for sentence in sent_words_list if sentence]


# ======================================== 인접 행렬 생성
G = nx.Graph()

# 모든 행에 대해 반복
for sent_word in final_sentences:
    # 문장 단위로 바이그램 생성 (같은 토큰이 쌍을 이루지 않도록)
    bigrams = [combo for combo in combinations(sent_word, 2) if combo[0] != combo[1]]
    for bigram in bigrams:
        if G.has_edge(*bigram):
            G[bigram[0]][bigram[1]]['weight'] += 1
        else:
            G.add_edge(bigram[0], bigram[1], weight=1)

# 자기 자신과의 연결(루프) 제거
G.remove_edges_from(nx.selfloop_edges(G))

# 네트워크 행렬 생성
adj_matrix = nx.to_pandas_adjacency(G, weight='weight')

# 전체 명사, 빈도수 튜플을 딕셔너리로 변경
frequency = dict(total_counts)

# 빈도수 상위 200개 단어 선택
sorted_frequency = sorted(frequency.items(), key=lambda x: x[1], reverse=True)[:200]
top_200_words = [word for word, freq in sorted_frequency]

# top_200_words 리스트와 인접 행렬의 인덱스를 교차하여 실제 존재하는 단어만 선택
existing_words = [word for word in top_200_words if word in adj_matrix.index]

# 인접 행렬에서 존재하는 단어들만 선택
filtered_adj_matrix = adj_matrix.loc[existing_words, existing_words]

# 인접 행렬을 CSV 파일로 저장
filtered_adj_matrix.to_csv(f'{name}_matrix_top200.csv', encoding='utf-8-sig', index=True)   # 한글 깨짐 방지: encoding='utf-8-sig' 추가
files.download(f'{name}_matrix_top200.csv')

NameError: name 'kss' is not defined

In [ ]:
# 네트워크 필터링: 엣지의 weight 값이 300 이상인 경우만 사용
filtered_edges = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] >= 0]

# 필터링된 노드들
filtered_nodenames = []
for u, v, d in G.edges(data=True):
    if d['weight'] >= 0:
        filtered_nodenames.append(u)
        filtered_nodenames.append(v)

filtered_nodenames = list(set(filtered_nodenames))

# 필터링된 서브그래프 생성
H = G.edge_subgraph(filtered_edges).copy()

# Degree Centrality 계산
degree_centrality = nx.degree_centrality(H)
node_size = [degree_centrality[node] * 10000 for node in H]

# 노드와 엣지 정보 엑셀 파일로 저장
# 노드 리스트 생성 (노드 이름과 중심성)
nodes_data = {
    'Id': list(H.nodes()),
    'Degree Centrality': [degree_centrality[node] for node in H.nodes()]
}
nodes_df = pd.DataFrame(nodes_data)
nodes_df.to_excel(f'{name}_nodes.xlsx', index=False)  # 노드 데이터를 엑셀로 저장

# 엣지 리스트 생성 (출발 노드, 도착 노드, 가중치)
edges_data = {
    'Source': [u for u, v in filtered_edges],
    'Target': [v for u, v in filtered_edges],
    'Weight': [G[u][v]['weight'] for u, v in filtered_edges]
}
edges_df = pd.DataFrame(edges_data)
edges_df.to_excel(f'{name}_edges.xlsx', index=False)  # 엣지 데이터를 엑셀로 저장

files.download(f'{name}_edges.xlsx')
files.download(f'{name}_nodes.xlsx')

print("노드와 엣지 데이터를 각각 'nodes.xlsx'와 'edges.xlsx' 파일로 저장했습니다.")